# Maximize Loan Amount in Lending Club

In this hypothetical case, we run a peer-to-peer lending platform called Lending Club. We want to maximize the loan amount to members. Let's use linear programming to optimize the loan amount.

1. [Data Preparation](#data)
2. [Objective Function - Weka](#weka)
3. [Objective Function - Python](#python)
4. [Constraints](#constraints)
5. [Linear Programming](#lp)

<a name="data"></a>
## 1. Data Preparation

We use a data set adapted from the original data at https://rdrr.io/cran/openintro/man/loans_full_schema.html

In [1]:
!wget https://raw.githubusercontent.com/rickysoo/lendingclub/main/loan50.csv

--2021-06-27 13:19:44--  https://raw.githubusercontent.com/rickysoo/lendingclub/main/loan50.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6958 (6.8K) [text/plain]
Saving to: ‘loan50.csv’

loan50.csv          100%[===================>]   6.79K  --.-KB/s    in 0s      

2021-06-27 13:19:44 (87.0 MB/s) - ‘loan50.csv’ saved [6958/6958]



In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

np.set_printoptions(suppress = True)

In [3]:
df = pd.read_csv('loan50.csv', index_col=0)
df.head()

,state,emp_length,term,homeownership,annual_income,verified_income,debt_to_income,total_credit_limit,total_credit_utilized,num_cc_carrying_balance,loan_purpose,loan_amount,grade,interest_rate,public_record_bankrupt,loan_status,has_second_income,total_income
1,NJ,3.0,60,rent,59000.0,Not Verified,0.557525,95131,32894,8,debt_consolidation,22000,B,10.90,0,Current,False,59000.0
2,CA,10.0,36,rent,60000.0,Not Verified,1.305683,51929,78341,2,credit_card,6000,B,9.92,1,Current,False,60000.0
3,SC,NaN,36,mortgage,75000.0,Verified,1.056280,301373,79221,14,debt_consolidation,25000,E,26.30,0,Current,False,75000.0
4,CA,0.0,36,rent,75000.0,Not Verified,0.574347,59890,43076,10,credit_card,6000,B,9.92,0,Current,False,75000.0
5,OH,4.0,60,mortgage,254000.0,Not Verified,0.238150,422619,60490,2,home_improvement,25000,B,9.43,0,Current,False,254000.0


Select numeric columns as variables.

In [4]:
features = ['emp_length', 'term', 'annual_income', 'debt_to_income', 'total_credit_limit', 'total_credit_utilized', 'num_cc_carrying_balance', 'interest_rate', 'public_record_bankrupt', 'has_second_income', 'total_income']
# features = ['annual_income', 'total_credit_limit', 'total_credit_utilized', 'total_income']
target = ['loan_amount']

df = df[features + target]

In [5]:
df.shape

(50, 12)

Check for missing values.

In [6]:
df.isnull().sum()

emp_length                 2
term                       0
annual_income              0
debt_to_income             0
total_credit_limit         0
total_credit_utilized      0
num_cc_carrying_balance    0
interest_rate              0
public_record_bankrupt     0
has_second_income          0
total_income               0
loan_amount                0
dtype: int64

Fix missing values and change one column from Boolean to numeric.

In [7]:
df['emp_length'].fillna(value = df['emp_length'].mean(), inplace = True)

df.loc[df['has_second_income'] == True, 'has_second_income'] = 1
df.loc[df['has_second_income'] == False, 'has_second_income'] = 0

In [8]:
df.head()

,emp_length,term,annual_income,debt_to_income,total_credit_limit,total_credit_utilized,num_cc_carrying_balance,interest_rate,public_record_bankrupt,has_second_income,total_income,loan_amount
1,3.000000,60,59000.0,0.557525,95131,32894,8,10.90,0,0,59000.0,22000
2,10.000000,36,60000.0,1.305683,51929,78341,2,9.92,1,0,60000.0,6000
3,4.895833,36,75000.0,1.056280,301373,79221,14,26.30,0,0,75000.0,25000
4,0.000000,36,75000.0,0.574347,59890,43076,10,9.92,0,0,75000.0,6000
5,4.000000,60,254000.0,0.238150,422619,60490,2,9.43,0,0,254000.0,25000


In [9]:
df.to_csv('loan50_small.csv', index = False)

<a name="weka"></a>
## 2. Objective Function - Weka

We use Weka to obtain the objective function.

In [ ]:
# import os
# import sys

# sys.path
# sys.path.append("/usr/lib/jvm/java-11-openjdk-amd64/bin/")
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64/"

# !apt-get install build-essential python3-dev
# !apt-get install python3-pil python3-pygraphviz
# !apt install openjdk-11-jdk

!pip3 install javabridge --no-cache-dir
!pip3 install python-weka-wrapper3 --no-cache-dir

     |████████████████████████████████| 1.4MB 6.9MB/s 
  Created wheel for javabridge: filename=javabridge-1.0.19-cp37-cp37m-linux_x86_64.whl size=1636591 sha256=2f341bdd6bc9ce2bb5d531561eb41daa941e414c0a6f85772031a6c9894ece65
  Stored in directory: /tmp/pip-ephem-wheel-cache-n9vzcqp0/wheels/3f/c3/06/e0ebe937afde69a1e691dde266bde0b67886c4edfa1479fb49
Successfully built javabridge
     |████████████████████████████████| 12.9MB 6.8MB/s 
  Created wheel for python-weka-wrapper3: filename=python_weka_wrapper3-0.2.3-cp37-none-any.whl size=11540519 sha256=41c89b708fbb1deb6c2fce1c18e2f9c04aaf7e7ae201bb4bc73e153f4825f698
  Stored in directory: /tmp/pip-ephem-wheel-cache-jajy3965/wheels/38/f9/79/fbc6f6db97bd80efc361ac947bcf082812669ef0928d55504f
Successfully built python-weka-wrapper3


In [ ]:
import weka.core.jvm as jvm
jvm.start()

In [ ]:
from weka.classifiers import Classifier
import weka.core.converters as converters

Load data into Weka.

In [ ]:
data = converters.load_any_file('loan50_small.csv')
data

Build a regression model using Linear Regression.

In [ ]:
from weka.classifiers import Classifier, Evaluation
from weka.core.classes import Random
data.class_is_last()   # set class attribute
classifier = Classifier(classname="weka.classifiers.functions.LinearRegression")
classifier.build_classifier(data)

Show the resulting objective function.

In [ ]:
print(classifier)

Evaluate the model.

In [ ]:
evaluation = Evaluation(data)
evaluation.crossvalidate_model(classifier, data, 10, Random(1))
print(evaluation.summary())

<a name="python"></a>
## 3. Objective Function - Python

We confirm the objective function by running it using Scikit-learn in Python.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

Take relevant variables generated in Weka as features.

In [ ]:
features = ['term', 'debt_to_income', 'total_credit_limit', 'total_credit_utilized', 'interest_rate']

In [ ]:
X = df[features]
y = df[target]

In [ ]:
X_ = PolynomialFeatures(degree = 1, include_bias = False).fit_transform(X)
X_.shape

In [ ]:
lr = LinearRegression().fit(X_, y)

In [ ]:
intercept, coefficients = lr.intercept_, lr.coef_
intercept, coefficients

The results match the ones produced in Weka.

### Objective Function

loan_amount =
    219.3865 * term +
 -11563.467  * debt_to_income +
      0.0145 * total_credit_limit +
      0.1476 * total_credit_utilized +
    428.0641 * interest_rate +
   -984.4325

<a name="constraints"></a>
## 4. Constraints

Explore the data to come up with reasonable constraints.

In [ ]:
df[features + target].describe()

In [ ]:
df['term'].plot(kind = 'hist', title = 'Term')

In [ ]:
df['debt_to_income'].plot(kind = 'hist', title = 'Debt-to-Income Ratio')

In [ ]:
df['total_credit_limit'].plot(kind = 'hist', title = 'Total Credit Limit')

In [ ]:
df['total_credit_utilized'].plot(kind = 'hist', title = 'Total Credit Utilized')

In [ ]:
df[['total_credit_limit', 'total_credit_utilized']].plot(kind = 'line', title = 'Credit Lines')

In [ ]:
df['interest_rate'].plot(kind = 'hist', title = 'Interest Rate')

In [ ]:
df['loan_amount'].plot(kind = 'hist', title = 'Loan Amount')

With the insights above, we can suggest the constraints below.

<a name="lp"></a>
## 5. Linear Programming

With the objective function and constraints, now we can optimize the function to calculate the maximum loan amount using simplex method. This is implemented in Python using SciPy library.

### Objective function

max (loan_amount) =
    219.3865 * term +
 -11563.467  * debt_to_income +
      0.0145 * total_credit_limit +
      0.1476 * total_credit_utilized +
    428.0641 * interest_rate +
   -984.4325
   
Note that max (loan_amount) = min (-loan_amount).

### Constraints

Due to implementation requirements in the scipy.optimize.linprog linear programming library, all inequalities in the constraints need to be in the "less than or equal" form.

**Term must be between 36 to 60 months**  
1. term >= 36  
(converted to -term <= -36)  

**Debt-to-income ratio must be between 0 to 1**  
2. debt_to_income >= 0   
(converted to -debt_to_income <= 0)  

**Total credit utilized must not exceed 80% of total credit limit. Both must be equal or more than 0.**  
3. total_credit_utilized <= 0.8 * total_credit_limit  
(converted to -0.8 * total_credit_limit + total_credit_utilized <= 0)  

4. total_credit_limit >= 0  
(converted to -total_credit_limit <= 0)  

5. total_credit_limit <= 800000  

6. total_credit_utilized >= 0  
(converted to -total_credit_utilized <= 0)  

**Interest rate must be between 5% to 20%**  
7. interest_rate >= 5  
(converted to -interest_rate <= 5)

8. interest_rate <= 20

**Additional Policies**

Interest rate needs to be commensurate to the risks inherent in term duration and debt-to-income ratio. The higher both these variables are, the higher the interest rate should be for the interest of lender.

9. interest_rate >= 0.1 * term + 3 * debt_to_income  
(converted to 0.1 * term + 3 * debt_to_income - interest_rate <= 0)

Loan duration needs to be commensurate to the risks inherent in debt-to-income ratio and total credit limit. It needs to be lower than a certain threshold to reduce the risk of the loan.

10. term <= 10 * debt_to_income + 0.0001 * total_credit_limit  
(converted to term - 10 * debt_to_income - 0.0001 * total_credit_limit <= 0)

Both additional policies have been tested in the data and found to allow for almost all loans except a few edge cases.

In [ ]:
from scipy.optimize import linprog

As max f(x) = -min f(-x), change the signs of all coefficients.

In [ ]:
obj = [-c for c in coefficients]
print(obj)

In [ ]:
lhs_ineq = [
    # term
    [-1, 0, 0, 0, 0],
    
    # debt_to_income
    [0, -1, 0, 0, 0],
    
    # total_credit_limit, total_credit_utilized
    [0, 0, -0.8, 1, 0],
    [0, 0, -1, 0, 0],
    [0, 0, 1, 0, 0],
    [0, 0, 0, -1, 0],
    
    # interest_rate
    [0, 0, 0, 0, -1],
    [0, 0, 0, 0, 1],

    # additional policies
    [0.1, 3, 0, 0, -1],
    [1, -10, -0.0001, 0, 0]
]

rhs_ineq = [-36, # term
            0, # debt_to_income
            0, 0, 800000, 0, # total_credit_limit, total_credit_utilized
            -5, 20, # interest_rate
            0, 0] # additional policies

opt = linprog(c = obj, A_ub = lhs_ineq, b_ub = rhs_ineq, method = "revised simplex")
print(opt)

Print the optimum variables.

In [ ]:
for var, x in zip(features, opt.x):
    print(f'Variable: {var}')
    print(f'Optimum: {x:.1f}')
    print('')

Get min f(x).

In [ ]:
obj_value = np.dot(obj, opt.x)[0]
obj_value

As max f(x) = -min f(x), change the sign and add intercept.

In [ ]:
max_loan = -obj_value + intercept[0]
print(f'Maximum loan amount = ${max_loan:.2f}')

Confirm the amount by multiplying the coefficients and the optimum variable values, and and adding up the intercept.

In [ ]:
answer = \
coefficients[0][0] * opt.x[0] + \
coefficients[0][1] * opt.x[1] + \
coefficients[0][2] * opt.x[2] + \
coefficients[0][3] * opt.x[3] + \
coefficients[0][4] * opt.x[4] + \
intercept[0]

answer

In [ ]:
answer == max_loan